In [13]:
pip install shapely


In [14]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import font_manager, rc
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl
from folium.features import GeoJson
from shapely.geometry import shape, Point
import json
from urllib.parse import urlparse, urlunparse, parse_qs, urlencode

# URL에 쿼리 파라미터를 추가하는 함수
def modify_url(url, params):
    url_parts = list(urlparse(url))
    query = dict(parse_qs(url_parts[4]))
    query.update(params)
    url_parts[4] = urlencode(query, doseq=True)
    return urlunparse(url_parts)

# 글꼴 경로 지정
font_path = "c:/Windows/Fonts/malgun.ttf"  # 윈도우에 설치된 맑은 고딕 폰트 경로

# 폰트 이름 얻어오기
font_name = font_manager.FontProperties(fname=font_path).get_name()

# matplotlib의 rc(run command) 기능을 이용하여 글꼴 설정
mpl.rc('font', family=font_name)

# 유니코드에서  음수 부호 설정
mpl.rc('axes', unicode_minus=False)

path = '../../../../../datasets/london_listings.csv'

raw = pd.read_csv(path)
df = raw.copy()

df = df[['id', 'host_id','host_is_superhost',
        'host_total_listings_count','neighbourhood_cleansed','room_type','accommodates','bathrooms','bedrooms',
        'beds','amenities','price','minimum_nights','maximum_nights','number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', 
        'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
        'review_scores_location', 'review_scores_value', 'reviews_per_month','listing_url','property_type','number_of_reviews_ltm','has_availability','last_review']]

df = df.rename(columns= {
    'id': '숙소_id',
    'host_id': '호스트_id',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수',
    'listing_url':'url',
    'property_type':'숙소_특징',
    'number_of_reviews_ltm':'12개월_리뷰수',
    'has_availability':'예약가능여부',
    'last_review':'마지막_리뷰'
})
df['위도']=raw['latitude']
df['경도']=raw['longitude']
print(len(df[df['리뷰수']==0]))
# 슈퍼호스트, 리뷰수 결측치 제거
print(f"처음 df : {len(df)}")
df = df[~df['슈퍼호스트'].isnull()]
print(f"슈퍼호스트 제거 : {len(df)}")
df = df[df['리뷰수']>2]
print(f"리뷰수 제거 : {len(df)}")

# 숙소가격 null값 제거
df= df[~df['숙소_가격'].isnull()]

#가격 앞 통화기호 제거
df['숙소_가격'] = df['숙소_가격'].replace('[\$,]', '', regex=True).astype(float)

# 유형 제거 
df = df[(df['숙소_유형']=='Entire home/apt') | (df['숙소_유형']=='Private room')]
print(f"숙소_유형 제거 : {len(df)}")

# 12개월 리뷰수 0 개 제거
df = df[df['12개월_리뷰수']!=0]
print(f"12개월_리뷰수 0개 제거 : {len(df)}")

# 욕실수, 침실수, 침대수 null값 제거
df = df.dropna(subset=['욕실수', '침실수', '침대수'])
print(f"욕실수, 침실수, 침대수 제거 후 : {len(df)}")

# 리뷰  null값 제거
df = df.dropna(subset=['숙소_정확성_리뷰점수','숙소_청결도_리뷰점수','숙소_체크인_리뷰점수','숙소_소통_리뷰점수','숙소_위치_리뷰점수','숙소_가격_리뷰점수'])
print(f"리뷰 결측치 제거 후 : {len(df)}")

# 예약 가능여부 f 버리기
df = df.dropna(subset='예약가능여부')
print(f"예약가능여부 : {len(df)}")

df_guest_prefer= df[(df['리뷰점수'] >= 4.9) & (df['리뷰수'] >= 5) & (df['슈퍼호스트'] == 't')]
df_non_guest_prefer = df[(df['슈퍼호스트'] == 'f') & (df['리뷰수'] >= 5)].sort_values('리뷰점수', ascending=True).head(len(df_guest_prefer))

24983
처음 df : 90852
슈퍼호스트 제거 : 90463
리뷰수 제거 : 50711
숙소_유형 제거 : 34776
12개월_리뷰수 0개 제거 : 31852
욕실수, 침실수, 침대수 제거 후 : 31781
리뷰 결측치 제거 후 : 31781
예약가능여부 : 31779


In [17]:
# # # CSV 파일 로드
# # df_metro = pd.read_csv('./파리_data/paris_stations.csv', delimiter=';')

# # # 'Coordonnées géographiques' 열을 공백으로 분리하여 새로운 열로 추가
# # # 데이터에서 쉼표도 제거 필요
# # df_metro[['위도', '경도']] = df_metro['Coordonnées géographiques'].str.replace(',', '').str.split(expand=True)

# # # 위도와 경도 데이터 타입을 float으로 변환
# # df_metro['위도'] = df_metro['위도'].astype(float)
# # df_metro['경도'] = df_metro['경도'].astype(float)

# # # 필요없는 'Coordonnées géographiques' 열 제거
# # df_metro.drop('Coordonnées géographiques', axis=1, inplace=True)

# # df_metro = df_metro.rename(columns= {
# #     'Identifiant station':'역_id',
# #     'Nom de la station' : '역이름',
# #     'Capacité de la station' : '역수용인원'
# # })

# # df_metro= df_metro[df_metro['역수용인원']!=0]

# # df_metro['역 이름 앞부분'] = df_metro['역이름'].apply(lambda x: x.split(' - ')[0])


# # # '역 이름 앞부분'으로 그룹화하고, 각 그룹의 위도와 경도 평균을 계산
# # station_groups = df_metro.groupby('역 이름 앞부분').agg({
# #     '위도': 'mean',
# #     # '경도': 'mean'
# # }).reset_index()

# # # 새로운 DataFrame 생성
# # df_station = station_groups.rename(columns={
# #     '역 이름 앞부분': '역이름'
# # })


In [18]:
df_station_check= pd.read_csv('./파리_data/Paris_Metro_Stations.csv')

In [19]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [20]:
# import requests

# api = 'mapapi넣을것'

# # 파리 지하철역 목록 (중복 제거)
# stations = [
#     "La Défense – Grande Arche", "Esplanade de la Défense", "Pont de Neuilly", "Les Sablons",
#     "Porte Maillot", "Argentine", "Charles de Gaulle – Étoile", "George V", "Franklin D. Roosevelt",
#     "Champs-Élysées – Clemenceau", "Concorde", "Tuileries", "Palais Royal – Musée du Louvre",
#     "Louvre – Rivoli", "Châtelet", "Hôtel de Ville", "Saint-Paul", "Bastille", "Gare de Lyon",
#     "Reuilly – Diderot", "Nation", "Porte de Vincennes", "Saint-Mandé", "Bérault", "Château de Vincennes",
#     "Porte Dauphine", "Victor Hugo", "Ternes", "Courcelles", "Monceau", "Villiers", "Rome",
#     "Place de Clichy", "Blanche", "Pigalle", "Anvers", "Barbès – Rochechouart", "La Chapelle",
#     "Stalingrad", "Jaurès", "Colonel Fabien", "Belleville", "Couronnes", "Ménilmontant",
#     "Père Lachaise", "Philippe Auguste", "Alexandre Dumas", "Avron", "Pont de Levallois – Bécon",
#     "Anatole France", "Louise Michel", "Porte de Champerret", "Péreire", "Wagram", "Malesherbes",
#     "Europe", "Saint-Lazare", "Havre – Caumartin", "Opéra", "Quatre Septembre", "Bourse", "Sentier",
#     "Réaumur – Sébastopol", "Arts et Métiers", "Temple", "République", "Parmentier", "Rue Saint-Maur",
#     "Gambetta", "Porte de Bagnolet", "Gallieni", "Porte de Clignancourt", "Simplon", "Marcadet – Poissonniers",
#     "Château Rouge", "Gare du Nord", "Gare de l'Est", "Château d'Eau", "Strasbourg – Saint-Denis",
#     "Étienne Marcel", "Les Halles", "Cité", "Saint-Michel", "Odéon", "Saint-Germain-des-Prés", "Saint-Sulpice",
#     "Saint-Placide", "Montparnasse – Bienvenüe", "Vavin", "Raspail", "Denfert-Rochereau", "Mouton-Duvernet",
#     "Alésia", "Porte d'Orléans", "Mairie de Montrouge", "Bagneux – Lucie Aubrac", "Bobigny – Pablo Picasso",
#     "Bobigny – Pantin – Raymond Queneau", "Église de Pantin", "Hoche", "Porte de Pantin", "Ourcq", "Laumière",
#     "Jourdain", "Place des Fêtes", "Télégraphe", "Porte des Lilas", "Mairie des Lilas", "Balard", "Lourmel",
#     "Boucicaut", "Félix Faure", "Commerce", "La Motte-Picquet – Grenelle", "École Militaire", "La Tour-Maubourg",
#     "Invalides", "Madeleine", "Richelieu – Drouot", "Grands Boulevards", "Bonne Nouvelle", "Filles du Calvaire",
#     "Saint-Sébastien – Froissart", "Chemin Vert", "Ledru-Rollin", "Faidherbe – Chaligny", "Montgallet", "Michel Bizot",
#     "Porte Dorée", "Porte de Charenton", "Liberté", "Charenton – Écoles", "École Vétérinaire de Maisons-Alfort",
#     "Maisons-Alfort – Stade", "Maisons-Alfort – Les Juilliottes", "Créteil – L'Échat", "Créteil – Université",
#     "Créteil – Préfecture", "Pointe du Lac", "Pont de Sèvres", "Billancourt", "Marcel Sembat", "Porte de Saint-Cloud",
#     "Exelmans", "Michel-Ange – Molitor", "Michel-Ange – Auteuil", "Jasmin", "Ranelagh", "La Muette", "Rue de la Pompe",
#     "Trocadéro", "Iéna", "Alma – Marceau", "Saint-Philippe du Roule", "Miromesnil", "Saint-Augustin",
#     "Havre – Caumartin", "Chaussée d'Antin – La Fayette", "Grands Boulevards", "Voltaire", "Charonne", "Rue des Boulets",
#     "Buzenval", "Maraîchers", "Porte de Montreuil", "Robespierre", "Croix de Chavaux", "Mairie de Montreuil",
#     "Boulogne – Pont de Saint-Cloud", "Boulogne – Jean Jaurès", "Chardon-Lagache", "Mirabeau", "Javel – André Citroën",
#     "Charles Michels", "Avenue Émile Zola", "Ségur", "Duroc", "Vaneau", "Sèvres – Babylone", "Mabillon", "Odéon",
#     "Cluny – La Sorbonne", "Maubert – Mutualité", "Cardinal Lemoine", "Jussieu", "Gare d'Austerlitz", "Campo-Formio",
#     "Place d'Italie", "Châtelet", "Hôtel de Ville", "Rambuteau", "Arts et Métiers", "Goncourt", "Belleville",
#     "Pyrénées", "Jourdain", "Aubervilliers – Front Populaire", "Porte de la Chapelle", "Marx Dormoy",
#     "Jules Joffrin", "Lamarck – Caulaincourt", "Abbesses", "Saint-Georges", "Notre-Dame de Lorette", "Trinité – d'Estienne d'Orves",
#     "Assemblée Nationale", "Solférino", "Rue du Bac", "Rennes", "Notre-Dame des Champs", "Falguière", "Pasteur",
#     "Volontaires", "Vaugirard", "Convention", "Porte de Versailles", "Corentin Celton", "Mairie d'Issy",
#     "Saint-Denis – Université", "Basilique de Saint-Denis", "Saint-Denis – Porte de Paris", "Carrefour Pleyel",
#     "Mairie de Saint-Ouen", "Garibaldi", "Porte de Saint-Ouen", "Guy Môquet", "La Fourche", "Liège",
#     "Saint-François-Xavier", "Duroc", "Gaîté", "Pernety", "Plaisance", "Porte de Vanves", "Malakoff – Plateau de Vanves",
#     "Malakoff – Rue Étienne Dolet", "Châtillon – Montrouge", "Les Agnettes", "Gabriel Péri", "Mairie de Clichy", "Porte de Clichy", "Brochant",
#     "Les Courtilles"
# ]

# # 중복 제거
# unique_stations = list(set(stations))

# # 각 역의 위도와 경도 정보를 가져오는 함수
# def get_geocode(station, api_key):
#     url = f"https://maps.googleapis.com/maps/api/geocode/json?address={station}, Paris&key={api_key}"
#     response = requests.get(url)
#     data = response.json()
#     if data['status'] == 'OK':
#         location = data['results'][0]['geometry']['location']
#         return (station, location['lat'], location['lng'])
#     else:
#         return (station, None, None)


# # 모든 역에 대한 위치 정보를 리스트로 수집
# results = [get_geocode(station, api_key) for station in unique_stations]

# # 결과를 데이터프레임으로 변환
# df_station_check = pd.DataFrame(results)

# # 결과 출력 및 CSV 파일로 저장
# print(df_station_check.head())  # 처음 몇 개의 데이터를 출력


# df_station_check.columns =['역이름','위도','경도']
# df_station_check.to_csv('./파리_data/Paris_Metro_Stations.csv', index=False)

In [21]:
df_station_check = pd.read_csv('./파리_data/Paris_Metro_Stations.csv')

In [22]:

# 파리 중심부의 위도와 경도
paris_latitude = 48.8566
paris_longitude = 2.3522

# 파리 지도 생성
map_paris = folium.Map(location=[paris_latitude, paris_longitude], zoom_start=12)

# 랜드마크 위치 데이터
landmarks = {
    "에펠탑": (48.8584, 2.2945),
    "루브르 박물관": (48.8606, 2.3376),
    "노트르담 대성당": (48.8529, 2.3500),
    "개선문": (48.8738, 2.2950),
    "가르니에 궁전": (48.8719, 2.3316),
    "사크레쾨르 대성당": (48.8867, 2.3431),
    "생트 샤펠": (48.8554, 2.3450),
    "알렉산드르 3세 다리": (48.8639, 2.3136),
    "마레 지구": (48.8575, 2.3588),
    "몽마르트": (48.8867, 2.3431),
    "팡테옹": (48.8463, 2.3460),
    "룩셈부르크 정원": (48.8462, 2.3371),
    "오르세 미술관": (48.8600, 2.3257),
    "샹젤리제 거리": (48.8696, 2.3079),
    "레 장발리드": (48.8565, 2.3126),
    "콩코르드 광장": (48.8656, 2.3211),
    "퐁피두 센터": (48.8606, 2.3522),
    "라 데팡스": (48.8900, 2.2400),
    "갤러리 라파예트": (48.8738, 2.3320),
    "그레뱅 박물관": (48.8718, 2.3422)
}

# 파리 경계 데이터 로드
with open('./파리_data/paris.geojson', 'r') as f:
    paris_boundary = json.load(f)

# shapely geometry 객체 생성
paris_area = shape(paris_boundary['features'][0]['geometry'])

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(map_paris)

# 각 랜드마크에 대한 마커 추가
for landmark, (lat, lng) in landmarks.items():
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='star', color='blue'),
        popup=landmark
    ).add_to(map_paris)

# 지하철 레이어 생성
subway_layer = FeatureGroup(name='Subway Stations', show=False)

# df_station_check 데이터를 이용해 지하철역 위치에 마커 추가
for idx, row in df_station_check.iterrows():
    if pd.notna(row['위도']) and pd.notna(row['경도']):
        folium.Marker(
            location=[row['위도'], row['경도']],
            icon=folium.Icon(icon='subway', prefix='fa', color='cadetblue'),  # 지하철 아이콘 적용
            popup=f"{row['역이름']} Station"
        ).add_to(subway_layer)

# 지하철 레이어를 지도에 추가
subway_layer.add_to(map_paris)

# 파리 경계 레이어 추가
GeoJson(
    paris_boundary_geojson,
    name='Paris Boundary',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(map_paris)


# df_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in df_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='green', icon='info-sign'),
        popup='Guest Prefer'
    ).add_to(marker_cluster)

# df_non_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in df_non_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup='Non-Guest Prefer'
    ).add_to(marker_cluster)
    
# 트램 레이어
tram_layer = FeatureGroup(name='Tram Stops')
geojson_data_tram = './파리_data/paris_tram.geojson'
GeoJson(geojson_data_tram, name='Tram Stops').add_to(tram_layer)
tram_layer.add_to(map_paris)

# 레이어 컨트롤 추가
LayerControl().add_to(map_paris)

# 지도를 HTML 파일로 저장
map_paris.save('paris_map.html')

# 지하철 역 주변 개수

In [23]:
import pandas as pd
from geopy.distance import geodesic

# 각 역에서 500m 이내에 있는 df_guest_prefer와 df_non_guest_prefer의 개수를 세는 함수
def count_nearby(df, station_lat, station_lon, radius=0.5):
    count = 0
    if pd.isna(station_lat) or pd.isna(station_lon):
        return count  # 입력된 역의 위도 또는 경도가 NaN이면 계산을 수행하지 않고 0을 반환
    station_coords = (station_lat, station_lon)
    for idx, row in df.iterrows():
        if pd.isna(row['위도']) or pd.isna(row['경도']):
            continue  # 데이터프레임의 행에 NaN이 있으면 이 행을 건너뛰기
        point_coords = (row['위도'], row['경도'])
        if geodesic(station_coords, point_coords).km <= radius:
            count += 1
    return count

# 결과를 저장할 리스트
results = []

# df_station_check의 각 역에 대해 반복
for idx, station in df_station_check.iterrows():
    station_name = station['역이름']
    lat = station['위도']
    lon = station['경도']
    
    # df_guest_prefer와 df_non_guest_prefer에서 각각의 카운트를 가져옴
    guest_count = count_nearby(df_guest_prefer, lat, lon)
    non_guest_count = count_nearby(df_non_guest_prefer, lat, lon)
    
    # 결과 저장
    results.append({
        '역이름': station_name,
        '위도': lat,
        '경도': lon,
        '게스트_선호': guest_count,
        '게스트_불호': non_guest_count
    })

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
print(result_df)


KeyboardInterrupt: 

# 랜드마크 주변 개수

In [ ]:
from geopy.distance import geodesic

# 특정 랜드마크와의 거리 계산 함수 정의
def count_accommodations_within_distance(df, landmark_coords, max_distance_km):
    count = 0
    for idx, row in df.iterrows():
        # 숙소 위치
        accommodation_location = (row['위도'], row['경도'])
        # 랜드마크와 숙소 간의 거리 계산
        distance = geodesic(landmark_coords, accommodation_location).km
        # 특정 거리 이내에 있으면 카운트
        if distance <= max_distance_km:
            count += 1
    return count

result1=[]
# 각 랜드마크와의 거리 계산
landmark_distances = {}
for landmark, coords in landmarks.items():
    count_prefer = count_accommodations_within_distance(df_guest_prefer, coords,0.5)  # 1km 이내
    count_non_prefer = count_accommodations_within_distance(df_non_guest_prefer, coords, 0.5)  # 1km 이내
    landmark_distances[landmark] = {'Guest Prefer': count_prefer, 'Non-Guest Prefer': count_non_prefer}
        # 결과 저장
    result1.append({
        '랜드마크': landmark,
        '위도': lat,
        '경도': lon,
        '게스트_선호': guest_count,
        '게스트_불호': non_guest_count
    })
    
# 결과를 데이터프레임으로 변환
result1_df = pd.DataFrame(result1)

# 결과 출력
print(result1_df)

           랜드마크         위도        경도  게스트_선호  게스트_불호
0           에펠탑  48.851757  2.401347      37      31
1       루브르 박물관  48.851757  2.401347      37      31
2      노트르담 대성당  48.851757  2.401347      37      31
3           개선문  48.851757  2.401347      37      31
4       가르니에 궁전  48.851757  2.401347      37      31
5     사크레쾨르 대성당  48.851757  2.401347      37      31
6         생트 샤펠  48.851757  2.401347      37      31
7   알렉산드르 3세 다리  48.851757  2.401347      37      31
8         마레 지구  48.851757  2.401347      37      31
9          몽마르트  48.851757  2.401347      37      31
10          팡테옹  48.851757  2.401347      37      31
11     룩셈부르크 정원  48.851757  2.401347      37      31
12      오르세 미술관  48.851757  2.401347      37      31
13      샹젤리제 거리  48.851757  2.401347      37      31
14       레 장발리드  48.851757  2.401347      37      31
15      콩코르드 광장  48.851757  2.401347      37      31
16       퐁피두 센터  48.851757  2.401347      37      31
17        라 데팡스  48.851757  2.401347      37  

# 지하철역 주변 개수

In [ ]:
import pandas as pd
from geopy.distance import geodesic

# 각 역에서 500m 이내에 있는 df_guest_prefer와 df_non_guest_prefer의 고유한 개수를 세는 함수
def count_nearby_unique(df, station_lat, station_lon, radius=0.5):
    unique_locations = set()  # 고유 위치를 저장할 세트
    count = 0
    if pd.isna(station_lat) or pd.isna(station_lon):
        return count  # 입력된 역의 위도 또는 경도가 NaN이면 계산을 수행하지 않고 0을 반환
    station_coords = (station_lat, station_lon)
    for idx, row in df.iterrows():
        if pd.isna(row['위도']) or pd.isna(row['경도']):
            continue  # 데이터프레임의 행에 NaN이 있으면 이 행을 건너뛰기
        point_coords = (row['위도'], row['경도'])
        point_coords_tuple = (round(row['위도'], 6), round(row['경도'], 6))  # 위도와 경도를 반올림하여 튜플 생성
        if point_coords_tuple not in unique_locations and geodesic(station_coords, point_coords).km <= radius:
            unique_locations.add(point_coords_tuple)  # 새로운 위치를 세트에 추가
            count += 1
    return count

# 결과를 저장할 리스트
results = []

# df_station_check의 각 역에 대해 반복
for idx, station in df_station.iterrows():
    station_name = station['역이름']
    lat = station['위도']
    lon = station['경도']
    
    # df_guest_prefer와 df_non_guest_prefer에서 각각의 카운트를 가져옴
    guest_count = count_nearby_unique(df_guest_prefer, lat, lon)
    non_guest_count = count_nearby_unique(df_non_guest_prefer, lat, lon)
    
    # 결과 저장
    results.append({
        '역이름': station_name,
        '위도': lat,
        '경도': lon,
        '게스트_선호': guest_count,
        '게스트_불호': non_guest_count
    })

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
print(result_df)

                   역이름         위도        경도  게스트_선호  게스트_불호
0     11 Novembre 1918  48.808895  2.538242       0       0
1         18 juin 1940  48.868807  2.185428       0       0
2           8 Mai 1945  48.784567  2.397904       0       0
3            Abbeville  48.879223  2.349147     103      97
4          Abbé Carton  48.827674  2.320918      30      36
...                ...        ...       ...     ...     ...
1171             Wurtz  48.826312  2.344586      30      13
1172    Youri Gagarine  48.783456  2.376529       0       0
1173       Yves Farges  48.957568  2.283450       0       0
1174  Édouard Vaillant  48.862655  2.208904       0       0
1175   Île de la Jatte  48.894036  2.266634       0       0

[1176 rows x 5 columns]
